A simple socket bridge to a streaming interface

In [ ]:
import socket
import requests
import json
from datetime import datetime, timedelta

# Wikimedia Recent Changes Stream URL
past_datetime = datetime.now() - timedelta(weeks=3)
since_timestamp = int(past_datetime.timestamp() * 1000)
URL = f'https://stream.wikimedia.org/v2/stream/recentchange?since={since_timestamp}'

# Socket configuration
HOST = "localhost"
PORT = 9998

def stream_to_socket():
    # Create a socket server
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen(1)
        print(f"Socket server started at {HOST}:{PORT}, waiting for connection...")

        conn, addr = server_socket.accept()
        print(f"Connection established with {addr}")

        # Open the HTTP stream
        with requests.get(URL, stream=True) as response:
            for line in response.iter_lines():
                if line and line.startswith(b'data:'):  # Filter lines containing event data
                    try:
                        event = json.loads(line[5:])  # Remove "data:" prefix and parse JSON
                        conn.sendall((json.dumps(event) + "\n").encode("utf-8"))
                    except json.JSONDecodeError:
                        continue

if __name__ == "__main__":
    stream_to_socket()
